In [ ]:
from IPython.core.display import HTML
import pandas as pd 

In [ ]:
spending_df = pd.read_csv('https://www.dropbox.com/s/ce9b47nzt3sx7y5/spending_10k.csv?dl=1', index_col="unique_id", dtype={"doctor_id":"object"})
spending_df.head(10)

### Overview


* In this section, we will tackle the handy `groupby` method.

* We also cover the split-apply-combine scheme to:

  * Aggregate data in each group
  * Transform data in each group
  * Filter the data in each group
  * Thin the data in each group

### `group_by` and `DataFrame` groups

* The `groupby()` method is used to group the data using values from one or more columns.

   * `groupby` takes as input one or more column labels, which it uses to group the data.

```python
df_1.groupby("X")
```

![](https://www.dropbox.com/s/86bi697t59zmkdn/groupby.png?dl=1)



### Identifying Groups from a GroupBy Object


```python
spending_df.groupby('specialty')
```

![](https://www.dropbox.com/s/bs8o34e4s7bdqa8/group_by_specialty.png?dl=1)

* The `groupby` method returns an object of type `DataFrameGroupBy.`
  * This is not a `DataFrame`, and does not, therefore, have the `DataFrame` methods discussed previously 




In [ ]:
x = spending_df.groupby('specialty')


In [ ]:
type(x)

In [ ]:
spending_by_specialty = spending_df.groupby('specialty')

addiction_med_group = spending_by_specialty.get_group("ADDICTION MEDICINE")
addiction_med_group

In [ ]:
x = pd.Series([1,2,3,4,5])

x / x.sum()

### `groupby` and Group-Specific Processing

* An ideal use-case for `groupby` consists of applying operations to each group independently.

* For instance, to compute the total spending by `specialty`, we need to:
  * Split the data by `specialty`.
  * Sum the total `spending` for each group.
  * Combine the sums for each group into a new `DataFrame`.




### Split-Apply-Combine Paradigm

* `groupby()` is often applied in the context of the data processing paradigm called "split-apply-combine".

  * **Split**: you need to split the data into chunks defined using one or more columns.
    * This is typically done using `groupby`.
  * **Apply**: apply some operation to the chunks generated.
    * Ex. Count the number of rows in each chunk, average the values for a specific column, etc.
  * **Combine**: combine the results of the applied operation into a new `DataFrame`.




### Split-Apply-Combine Example

![](https://www.dropbox.com/s/aecufw3mfu2mlah/split_apply_combine_example.png?dl=1)

* The type of Split-Apply-Combine applied here is referred to as aggregation.
  * Aggregations refer to any operation that aggregates (reduces) group data to a single value.

### The 3 ( or 3  $\frac{1}{2}$) Classes of Opearations on Groups


* Three are 3 formal classes of split-apply-combine operations that can be applied to group data.

  * I include a variant ($\frac{1}{2}$ a class) which I think is useful to better classify split-apply-combine operations.


1\.$~~$__Aggregations__ generate a single value for each group
   * Ex. Sum the spending by specialty
  
2\.$~~$ __Transformations__ convert the data and generate a group of the same size as the original group.
   * Ex. Convert the currency by country for some datasets that contains medication cost by country.

3\.$~~$ __Filters__ retain or discard a group based on group-specific boolean computations.
   * Ex. drop specialty if the sum of spending is below some threshold

3$\frac{1}{2}$\.$~$"__Thinning__" drops entries in a group based on some defined logic.
  * Filter out values in a group that are 3 standard deviations above or below the mean.
  


### Aggregations

- __Aggregations__ aggregate the data in each group, i.e., they reduce the data in each group to a single value. 

  * This includes, for instance, computing group sums, means, maximums, minimums, _etc_.



![](https://www.dropbox.com/s/9q54na9szs5syi5/aggregate.png?dl=1)



### Transforming Group Data

* Transform the data in a group-specific way.

  *  Ex. for specialty, we want to transform the column `nb_beneficiaries` into the values small, large or medium, depending on whether the `nb_beneficiaries` value is, respectively, `-2 * std` below the mean, `+2 * std` larger than the mean or withing `+/-2 * std` of the mean.


   *  The number of entries per group resulting from a transformation is the same as the number of entries in the group before the transformation.



- The diagram below shows an example where the data in column "Y" in transformed by dividing it by the group mean.

![](https://www.dropbox.com/s/nf8lg0lqk3yxf7k/transform_2.png?dl=1)


### Filtering Group Data

* Consist of dropping or retaining that group in a way that depends on a group-specific computation that returns `True` or `False`. 

* For instance, we can filter specialties that don't have enough entries or for which the mean `spending` if below a certain threshold.
  * Groups are either retained or discarded. Groups that are retained are unmodified.


- The diagram below shows an example where groups are filtered if their sum for column `Y` is less than 10.

![](https://www.dropbox.com/s/ncmv2xsupjok7va/filter.png?dl=1)

### Thinning Group Data

* Consist of reducing the number of entries using a group-specific operation.

* Thinning can be useful to sub-sample the data at the group level or returning the top `n` entries in each group, etc. 

  * As opposed to aggregating functions, thinning does not have to reduce the group into a single entry; although it could

    
![](https://www.dropbox.com/s/m4p4f5nk55w2ni2/thin.png?dl=1) 


### Aggregating the Data Using `groupby`

* Aggregation is commonly used to compute summary statistics on each of the groups.

* Some of the interesting/important summary aggregation methods `DataFrameGroupBy` objects are:

|Methods           |        Decription                              |
|:-----------------|:-----------------------------------------------|
| `mean`, `median` | Computes the mean and the median in each group | 
| `min` , `max`    | computes the min and max in each group         | 
| `size`           | computes the number of values in each group    | 




In [ ]:
spending_df.head()

##### Aggregating the Data Using `groupby` Cont'd 


- The functions above all use the same syntax:
 
```python
spending_df.groupby('specialty').sum()
# or
spending_df.groupby('specialty').min()
```


In [ ]:
spending_df.groupby('specialty').sum().head(10)



### Applying Functions to Group Columns

- The method called `agg` can be used where complex or custom aggregation logic is required.
 The method `agg` takes a function (or a list of functions) and uses it (them) to aggregate the group's colum(s)

- Example, we can use `sum_spending_CAD` to return the sum of the spending in Canadian Dollars.



```python
def sum_spending_CAD(x):
    return x.sum() * 1.32

spending_by_specialty['spending'].agg(sum_spending_CAD)
```


* `agg` can either:
  * take a dictionary of functions to aggregate on.
    * Required for aggregating more than one column 

    ```python 
    spending_by_specialty.agg({'nb_beneficiaries' :sum,
                               'spending' : sum_spending_CAD)
    ```

  * `agg` can take a list of function to apply to each column functions to aggregate on.
  
    `
    spending_by_specialty.agg([min,max,sum])
    `

In [ ]:
def sum_spending_CAD(x):
    return x.sum() * 1.32

# format
(
    spending_by_specialty.agg({ 'nb_beneficiaries': sum, 'spending': sum_spending_CAD })
                         .head()
)



In [ ]:
spending_by_specialty.get_group("ADDICTION MEDICINE")

In [ ]:
# note that sum, min and max here are functions
spending_by_specialty['spending'].agg([sum, min, max]).head()

In [ ]:
spending_by_specialty.agg({'nb_beneficiaries' :min,
                           'spending' : max}).head()


In [ ]:
spending_by_specialty.agg({'nb_beneficiaries' :[min, sum],
                           'spending' : max}).head()


### Transforming the Data in `groupby`

- As opposed to aggregations, which reduce the data into a single value, transformations modify the data but don't change the `shape` (dimension) of the groups

- Transformations are useful for applying operations that are group specific



### Transforming the Data in `groupby` Cont'd


- The example below computes the percent contribution of each entry to each specialty by applying a transformation that normalizes the entry's spending over the total spending in that specialty. 

![](https://www.dropbox.com/s/xwomvq1cs90jpg1/transform_spending.png?dl=1)


In [ ]:
spending_by_specialty["spending"].get_group("ADDICTION MEDICINE")

### Applying a Transformation

- Applying a transformation is done using the method called `transform`.


- The method `transform` takes as input a function name, which it calls on each group of the `DataFrameGroupBy` object

In [ ]:
# i=0
# global i
# i+=1 
# print(type(x))


def my_function(x):

    return (x   / x.sum() ) * 100
    


spending_df["spending_pct"] = spending_by_specialty['spending'].transform(my_function)


In [ ]:
spending_df[spending_df['specialty'] == "ADDICTION MEDICINE"]


In [ ]:
spending_df.sort_values(['specialty', 'spending_pct'], ascending=[True, False]).head(10)

### More complex Transformations

* As noted above, drugs are still duplicated across `doctor_ids` within the same `specialty.`

  *  ex. FLUTICASONE/SALMETEROL is prescribed by at least 3 doctors

- To see the percent spending by `drug` column, we need to group on both the `specialty` and the `medication` and then sum the `spending_pct` computed previously

```python
medication_spendng_pct =  spending_df.groupby(["specialty", "medication"])["spending_pct"].sum()
```



In [ ]:
medication_spendng_pct.head()

In [ ]:
medication_spendng_pct =  spending_df.groupby(["specialty", "medication"])["spending_pct"].sum()
print(type(medication_spendng_pct))
print("\n" + "*" * 35 + "\n")
print(medication_spendng_pct.index)



In [ ]:
import string
import random

print(string.ascii_letters)
print("\n" + "*" * 52 + "\n")

lc_letters = list(string.ascii_letters[:26])
print(lc_letters)

print("\n" + "*" * 52 + "\n")

print(random.sample(lc_letters, 6))


In [ ]:
x = pd.Series(random.sample(lc_letters, 6), index=[1,2,3,4,5,6])
x.head()


In [ ]:
print(x.index)

print("\n" + "*" * 45 + "\n")

print(x[1])

In [ ]:
medication_spendng_pct.index

In [ ]:
medication_spendng_pct[('ADDICTION MEDICINE', 'BUSPIRONE HCL')]

In [ ]:
medication_spendng_pct[('ADDICTION MEDICINE', )]

In [ ]:
medication_spendng_pct[('ALLERGY/IMMUNOLOGY', )]

### More complex Transformations- cont'd

* The multiindex is sometime inconvenient to work with
    * Makes it hard to sort on `speciality` and `spending_pct` as we did earlier

* We can reset (drop) the index using the method `reset_index`
  * allows us to sort on `specialty` and `spending_pct` as we did earlier





In [ ]:
spending_df.groupby(["specialty", "medication"])["spending_pct"].sum().head()

In [ ]:
medication_spendng_pct = spending_df.groupby(["specialty", "medication"])["spending_pct"].sum().reset_index()
medication_spendng_pct.head()


In [ ]:
medication_spendng_pct.sort_values(["specialty", "spending_pct"], ascending=[True, False]).head(10)

### Filtering Groups

- Filtering a group is done using the method called `filter`


- The method `filter` takes as input a function name, which it calls on each group of the `DataFrameGroupBy` object
  - The function must return either `True` or `False`.
  - Groups for which the function returns `False` are dropped.


- The resulting` DataFrame` has its entries in the same order as the original `DataFrame`.
 


In [ ]:
spending_df['specialty'].unique()

In [ ]:

def filter_on_spending(x):
    return x['spending'].sum() > 50_000

high_spending_df = spending_df[["specialty", 'spending']].groupby('specialty').filter(filter_on_spending)



In [ ]:
high_spending_df['specialty'].unique() 


### Thinning Groups

* Thinning the data consist in reducing the number of entries in a group

* As opposed to aggregating functions, thinning does not have to reduce the group into a single entry
  * Although it could reduce it to a single entry


* Thinning can be used, for instance, to return only the top 3 entries in each category, or to randomly sample a small subset of entries from each category

### Thinning Methods and `apply`

- `pandas` offers a few methods for thinning the data.
  - Ex. `nlargest`, `nsmallest`, etc.
    
    
- However, thinning  is most often carried out using a method  called `apply.` 



- The  method `apply` takes as input a function name, which it calls on each group of the `DataFrameGroupBy` object.


In [ ]:
spending_by_specialty['spending'].nlargest(2)

In [ ]:
spending_by_specialty['spending'].nsmallest(3)

### Sub-sampling a DataFrame


- This is necessary to maintain group composions.

- This can be achived using the DataFrame mthod called `sample.` 

  - Two parameters are relevant in this scenario,`n` the number of samples to randomly select or `frac` a portion of the data to retun
  - We are interested the latter

```python
 spending_df.sample(frac=0.001)
```


In [ ]:
# return 0.01% of the data, i.e 10 entries
spending_df.sample(frac=0.01).head()


In [ ]:
# return 0.01% of the data, i.e 10 entries
spending_df.sample(n=10) 

In [ ]:
# We sample only 10% of the Data in each category

def sample_10p(x):
    return x.sample(frac=0.1)
    
    
# spending_by_specialty.apply(sample_10p).head()
spending_df.groupby('specialty').apply(sample_10p).head()

In [ ]:

print(spending_by_specialty.get_group("CARDIAC ELECTROPHYSIOLOGY").shape)
print(spending_by_specialty.get_group("ANESTHESIOLOGY").shape)
print(spending_by_specialty.get_group("CARDIOLOGY").shape)


In [ ]:
subsampled_spending_df = spending_by_specialty.apply(sample_10p)

print(subsampled_spending_df.loc["CARDIAC ELECTROPHYSIOLOGY"].shape)

print(subsampled_spending_df.loc["ANESTHESIOLOGY"].shape)

print(subsampled_spending_df.loc["CARDIOLOGY"].shape)




In [ ]:
subsampled_spending_df.head()